In [96]:
import pandas as pd
import os
import numpy as np

In [144]:
foldername = 'D1'
data_file_folder = f'./{foldername}'
df = []

for file in os.listdir(data_file_folder):
    if file.endswith('.xlsx'):
        sheet_name, extension = file.split('.')
        file = f'{data_file_folder}/{file}'
        df.append(pd.read_excel(file, sheet_name = sheet_name, usecols = 'A:J'))
df_master = pd.concat(df, axis=0)

In [145]:
df_master.head()

,Receiving Committee,Filing Period,Contribution Date,Contributor Name,Contributor Address,Contributor Type,Contribution Type,Contribution Amount,Employer Name,Employer Occupation
0,"Walsh, Liz (Elizabeth) for Howard County",2022 Gubernatorial Pre-Primary1,2022-05-03,"Ponnuri, Sunita","4616 Broken Lute Way,Howard, Ellicott City, MD...",Individual - Matching fund request,Check,50.00,NaN,NaN
1,"Walsh, Liz (Elizabeth) for Howard County",2022 Gubernatorial Pre-Primary1,2022-05-13,"Beck, Claudia","9746 Gingerwood Drive,Howard, Ellicott City, M...",Individual - Matching fund request,Credit Card,250.00,NaN,NaN
2,"Walsh, Liz (Elizabeth) for Howard County",2022 Gubernatorial Pre-Primary1,2022-05-23,"Boyter, Angie","3914 MACALPINE RD,Howard, Ellicott City, MD 21042",Individual - Matching fund request,Credit Card,250.00,NaN,NaN
3,"Walsh, Liz (Elizabeth) for Howard County",2022 Gubernatorial Pre-Primary1,2022-05-15,"Callens, Laura","2960 Normandy Dr,Howard, Ellicott City, MD 21043",Individual - Matching fund request,Credit Card,5.00,NaN,NaN
4,"Walsh, Liz (Elizabeth) for Howard County",2022 Gubernatorial Pre-Primary1,2022-05-13,"Greco, Megan","11320 Cotswold Spring Farm Lane,Howard, Ellico...",Individual - Matching fund request,Credit Card,103.45,NaN,NaN


In [146]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 592 entries, 0 to 116
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Receiving Committee  592 non-null    object        
 1   Filing Period        592 non-null    object        
 2   Contribution Date    592 non-null    datetime64[ns]
 3   Contributor Name     592 non-null    object        
 4   Contributor Address  592 non-null    object        
 5   Contributor Type     592 non-null    object        
 6   Contribution Type    592 non-null    object        
 7   Contribution Amount  592 non-null    float64       
 8   Employer Name        220 non-null    object        
 9   Employer Occupation  288 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 50.9+ KB


In [147]:
candidate_name = df_master['Receiving Committee'].unique().tolist()
donor_name = df_master['Contributor Name'].unique().tolist()
contributions = df_master['Contribution Amount'].unique().tolist()
filing_period = df_master['Filing Period'].unique().tolist()

In [148]:
mask=df_master['Receiving Committee'].isin(candidate_name)
## mask2 = df_master['Contributor Name'].isin(donor_name)

In [149]:
df_contribution_sum = df_master[mask].groupby(by = ['Contributor Name', 'Receiving Committee']).sum()[['Contribution Amount']]
df_contribution_count = df_master[mask].groupby(by = ['Contributor Name','Receiving Committee']).count()[['Contribution Amount']]
df_contribution_count = df_contribution_count.rename(columns = {'Contribution Amount':'No of Contributions'})

In [150]:
df_grouped = [df_contribution_sum, df_contribution_count]

In [151]:
df_grouped = pd.concat(df_grouped, axis = 1)
df_grouped = df_grouped.sort_values(by=['No of Contributions'], ascending = False)

In [152]:
group_by = df_master[mask].groupby(['Contributor Name', 'Receiving Committee']).agg({'Contribution Amount':'sum', 'Contributor Name':'count'}).rename(columns={'Contributor Name':'No of Contributions'}).sort_values(by=['No of Contributions'], ascending = False)
group_by


,,Contribution Amount,No of Contributions
Contributor Name,Receiving Committee,,
"Walsh, Elizabeth","Walsh, Liz (Elizabeth) Committee to Elect",2330.90,21
"Wagner, Joseph","Kurtom, Neveen Friends of",5638.10,13
"Popp, Dawn","Walsh, Liz (Elizabeth) Committee to Elect",1768.13,12
"Kepnes, Kimberly","Walsh, Liz (Elizabeth) Committee to Elect",1352.06,10
"Farrell, Katherine","Walsh, Liz (Elizabeth) Committee to Elect",386.00,9
...,...,...,...
"Hohmann, Lars","Walsh, Liz (Elizabeth) for Howard County",10.53,1
"Hoagland, Gina","Walsh, Liz (Elizabeth) Committee to Elect",10.00,1
"Himel, James","Kurtom, Neveen Friends of",500.00,1


In [153]:
f = {'Contributor Name': 'nunique', 'Contribution Amount':'np.sum'}
df_contribution = df_master[mask].groupby(['Contributor Name', 'Receiving Committee'])
#df_contribution_sum = df_contribution.agg(f)
#df_contribution_sum

In [154]:
df_grouped = df_master[mask].groupby(['Contributor Name', 'Receiving Committee']).agg({'Contribution Amount':["sum"], 'Contributor Name':["count"]})
df_grouped.columns=['Total Contribution', 'No of Contributions']
df_grouped

,,Total Contribution,No of Contributions
Contributor Name,Receiving Committee,,
"Achey, Toni","Walsh, Liz (Elizabeth) Committee to Elect",24.00,1
"Adams, William","Walsh, Liz (Elizabeth) Committee to Elect",100.00,3
"Adler, Delphine","Kurtom, Neveen Friends of",200.00,1
"Adler, Steve","Kurtom, Neveen Friends of",100.00,1
"Ainslie, Katie","Walsh, Liz (Elizabeth) Committee to Elect",10.00,1
...,...,...,...
"Zhang, Li","Walsh, Liz (Elizabeth) for Howard County",100.00,1
"de Melo, Lucas","Walsh, Liz (Elizabeth) for Howard County",250.00,1
"langley, James","Kurtom, Neveen Friends of",1000.00,1


In [155]:
developer_list = pd.read_csv('developercrossreference.csv').drop_duplicates()
developer_list = developer_list.dropna()
developer_list['Developer/Developer Affiliated']=developer_list['Developer/Developer Affiliated'].str.lower()
developer_id = developer_list['Developer/Developer Affiliated']=='yes'
developer_list = developer_list[developer_id]
developer_list=developer_list.reset_index()
developer_list = developer_list.drop(columns = 'index')
developer_filter=df_master['Contributor Name'].isin(developer_list['Contributor Name'])
developer_contributions = df_master[developer_filter]

In [156]:
developer_contributions

,Receiving Committee,Filing Period,Contribution Date,Contributor Name,Contributor Address,Contributor Type,Contribution Type,Contribution Amount,Employer Name,Employer Occupation
1,"Kurtom, Neveen Friends of",2022 Annual,2021-11-02,"O""Doherty, Damian","508 W. Joppa Road, Towson, MD 21204",Individual,Ticket Purchases Credit Card,950.0,KO Public Affairs,Other
2,"Kurtom, Neveen Friends of",2022 Annual,2021-10-22,"Rubin, Marc","11229 Welland St, North Potomac, MD 20878",Individual,Ticket Purchases Credit Card,250.0,NaN,NaN
4,"Kurtom, Neveen Friends of",2022 Annual,2021-06-27,"Wagner, Joseph","6363 Rowanberry Drive, Elkridge, MD 21075",Individual,Check,250.0,"HBP, Inc.",Other
6,"Kurtom, Neveen Friends of",2022 Annual,2021-10-22,"Guckert, John","9900 Franklin Square Drive Suite H, Baltimore,...",Individual,Ticket Purchases Credit Card,250.0,NaN,NaN
7,"Kurtom, Neveen Friends of",2022 Annual,2021-08-20,"Corridor Square, LLC","6800 Deeepath Road, Suite 100, Elkridge, MD 21075",Business/Group/Organization,Check,1000.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
106,"Kurtom, Neveen Friends of",2022 Annual,2022-01-10,"Breeden, Kevin","P.O. Box 77, Casanova, VA 20139",Individual,Credit Card,500.0,Self-Employed,Legal
111,"Kurtom, Neveen Friends of",2022 Annual,2021-10-27,"Tabuteau, Richard","300 Dunkirk Road, Baltimore, MD 21212",Individual,Credit Card,250.0,NaN,NaN
112,"Kurtom, Neveen Friends of",2022 Annual,2022-01-12,Sung Hwang & Kim LLP,"9256 Bendix Road, Suite 109, Columbia, MD 21045",Business/Group/Organization,Credit Card,200.0,NaN,NaN
114,"Kurtom, Neveen Friends of",2022 Annual,2021-09-12,"Clark, Robert","10 Clarion Court, Cockeysville, MD 21030",Individual,Credit Card,100.0,"Law Office of Robert W. Clark, Jr.",Legal


In [157]:
df_grouped = df_master[mask].groupby(['Contributor Name', 'Receiving Committee'], as_index=False).agg({'Contribution Amount':["sum"], 'Contributor Name':["count"]})
df_grouped.reset_index()

,index,Receiving Committee,Contribution Amount,Contributor Name
,,,sum,count
0,0,"Walsh, Liz (Elizabeth) Committee to Elect",24.00,1
1,1,"Walsh, Liz (Elizabeth) Committee to Elect",100.00,3
2,2,"Kurtom, Neveen Friends of",200.00,1
3,3,"Kurtom, Neveen Friends of",100.00,1
4,4,"Walsh, Liz (Elizabeth) Committee to Elect",10.00,1
...,...,...,...,...
445,445,"Walsh, Liz (Elizabeth) for Howard County",100.00,1
446,446,"Walsh, Liz (Elizabeth) for Howard County",250.00,1
447,447,"Kurtom, Neveen Friends of",1000.00,1


In [158]:
import plotly.express as px
developer_list = pd.read_csv('developercrossreference.csv')
developer_list = developer_list.dropna()
developer_list['Developer/Developer Affiliated']=developer_list['Developer/Developer Affiliated'].str.lower()
developer_id = developer_list['Developer/Developer Affiliated']=='yes'
developer_list = developer_list[developer_id]
developer_list=developer_list.reset_index()
developer_list = developer_list.drop_duplicates(subset=['Contributor Name'])
developer_list = developer_list.drop(columns = 'index')

developer_filter=df_master['Contributor Name'].isin(developer_list['Contributor Name'])
df_grouped = df_master[developer_filter].groupby(['Contributor Name', 'Receiving Committee']).agg({'Contribution Amount':["sum"], 'Contributor Name':["count"]})
df_grouped.columns=['Total Contribution', 'No of Contributions']
df_grouped_for_plot = df_grouped
table_cols = df_grouped_for_plot.reset_index().sort_values(by=['Total Contribution'], ascending = False).head(10)
fig = px.bar(table_cols, x=table_cols['Contributor Name'], y=table_cols['Total Contribution'])

In [159]:
table_cols

,Contributor Name,Receiving Committee,Total Contribution,No of Contributions
38,"Wagner, Joseph","Kurtom, Neveen Friends of",5638.1,13
22,"O""Doherty, Damian","Kurtom, Neveen Friends of",2850.0,3
35,The Howard Research and Development Corporation,"Kurtom, Neveen Friends of",2000.0,2
15,"Hyland, Timothy","Kurtom, Neveen Friends of",1700.0,3
17,"Kelehan, Kevin","Kurtom, Neveen Friends of",1050.0,2
28,"S.E.H. Excavating, Inc.","Kurtom, Neveen Friends of",1000.0,1
14,"Heath, Jeremy","Kurtom, Neveen Friends of",1000.0,1
10,"Griffin, Corey","Kurtom, Neveen Friends of",1000.0,2
40,"langley, James","Kurtom, Neveen Friends of",1000.0,1
29,SCL Consulting LLC,"Kurtom, Neveen Friends of",1000.0,1


In [175]:
contribution_by_filing_period = df_master[mask].groupby(['Receiving Committee','Filing Period'])
contribution_by_filing_period=contribution_by_filing_period.agg(
    TotalContribution =('Contribution Amount','sum')).reset_index()
#contribution_by_filing_period.columns=['Total Contribution']
contribution_by_filing_period

,Receiving Committee,Filing Period,TotalContribution
0,"Kurtom, Neveen Friends of",2022 Annual,41048.48
1,"Walsh, Liz (Elizabeth) Committee to Elect",2018 Gubernatorial Post-General,7052.43
2,"Walsh, Liz (Elizabeth) Committee to Elect",2018 Gubernatorial Pre-General1 Report,4940.13
3,"Walsh, Liz (Elizabeth) Committee to Elect",2018 Gubernatorial Pre-General2 Report,12644.39
4,"Walsh, Liz (Elizabeth) Committee to Elect",2018 Gubernatorial Pre-Primary1,3717.87
5,"Walsh, Liz (Elizabeth) Committee to Elect",2018 Gubernatorial Pre-Primary2,1362.55
6,"Walsh, Liz (Elizabeth) Committee to Elect",2018 Gubernatorial Spring Report,5698.72
7,"Walsh, Liz (Elizabeth) Committee to Elect",2019 Annual,312.00
8,"Walsh, Liz (Elizabeth) Committee to Elect",2020 Annual,112.00
9,"Walsh, Liz (Elizabeth) Committee to Elect",2022 Annual,493.00


In [181]:
fig = px.bar(contribution_by_filing_period, x = contribution_by_filing_period['Filing Period'], y=contribution_by_filing_period['TotalContribution'],color=contribution_by_filing_period['Receiving Committee'])

In [182]:
fig

In [188]:
import streamlit as st
v=st.columns(3)

In [189]:
v[0]

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [221]:
trial_mask = df_master['Receiving Committee']=='Walsh, Liz (Elizabeth) for Howard County'
df_master[mask][trial_mask].shape

(163, 10)